In [1]:
"""docstring :)"""

'docstring :)'

In [2]:
import json
import requests
import pandas as pd
from riotwatcher import LolWatcher, ApiError
import numpy as np
import matplotlib.pyplot as plt
# from time import sleep

with open('data/api-key.txt', 'r') as api:
    API_KEY = api.read()
lol_watcher = LolWatcher(API_KEY)

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4)"+
                    " AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135"+
                    " Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": API_KEY
}

CHAMP_ID = 64
TESTMATCH = "4748107995"

np.random.seed(7)

matches = pd.read_csv("data/clid-matches.csv")

the time stamp 44111 is 44 seconds and 111 milliseconds

In [3]:
matches

,platformId,gameId,champion,queue,season,timestamp,role,lane
0,KR,4758318885,64,420,13,1604330054317,NONE,JUNGLE
1,KR,4758080001,64,420,13,1604321575669,NONE,JUNGLE
2,KR,4757537954,64,420,13,1604308057379,NONE,JUNGLE
3,KR,4756152547,64,420,13,1604233077276,NONE,JUNGLE
4,KR,4755546321,64,420,13,1604226697905,NONE,JUNGLE
...,...,...,...,...,...,...,...,...
518,KR,3432455664,64,420,12,1542973849243,NONE,JUNGLE
519,KR,3430594315,64,420,12,1542822782724,NONE,JUNGLE
520,KR,3420476831,64,420,12,1542113321062,NONE,JUNGLE
521,KR,3419108414,64,420,11,1542007670086,NONE,JUNGLE


In [4]:
def get_match(match_id, region="kr"):
    """matchid is str, returns match info which is in json form, and returns clid's participant id for the game"""
    match_request = requests.get("https://"+region+".api.riotgames.com/lol/match/v4/matches/"+match_id,
                                 headers=HEADERS)
    match_json = json.loads(match_request.text)
    for participant in match_json["participants"]:
        if participant["championId"] == CHAMP_ID:
            player_participant_id = participant["participantId"] 
    return match_json, player_participant_id

In [5]:
def get_timeline(match_id, ppi, region="kr", give_json=True):
    url = ["https://", ".api.riotgames.com/lol/match/v4/timelines/by-match/"]
    url = "".join([url[0], region, url[1], match_id])
    timeline_res = requests.get(url, headers=HEADERS)

    timeline_obj = json.loads(timeline_res.text)
    
    flattened_frames = []
    j=0
    for frame in timeline_obj["frames"]:
        timestamp = frame["timestamp"]
        for value in frame["participantFrames"].values():
            try:
                value["x"] = value["position"]["x"]
                value["y"] = value["position"]["y"]
            except:
                value["x"] = np.nan
                value["y"] = np.nan

        frame_df = pd.DataFrame.from_dict(frame["participantFrames"])#.values.flatten()
        try:
            frame_df = frame_df.drop(["position", "teamScore", "dominionScore"])
        except KeyError:
            pass
        if j==0:
            curr = frame_df
            j=1
        flatten_frame = frame_df.values.flatten()
        flattened_frames.append(np.append([timestamp],
                                          flatten_frame))
        
    cols = ["".join([str(i), col]) for col in frame_df.index for i in range(10)]
    
    flattened_df = pd.DataFrame(flattened_frames, columns = ["timestamp"] + cols)
    
    dropcolslem = ["participantId", "xp", "minionsKilled"]
    dropcolsnames = [colname for colname in cols if colname[1:] in dropcolslem]
    
    flattened_df = flattened_df.drop(dropcolsnames, axis=1)
    ppi = ppi-1
    #print(ppi)
    player_pos_col = [str(ppi)+"x", str(ppi)+"y"]
    try:
        flattened_df["player_x"] = flattened_df[player_pos_col[0]]
        flattened_df["player_y"] = flattened_df[player_pos_col[1]]
    except Exception as e:
        print(flattened_df.columns)
        print("ppi err",ppi)
        raise e
    flattened_df = flattened_df.drop(player_pos_col, axis=1)
    if give_json:
        return {"df": flattened_df, "json": timeline_obj}
    else:
        return {"df": flattened_df}
    

In [6]:
def get_n_timelines(match_df, start_index, end_index, give_json=True, region="kr"):
    matches = {}
    dfs = {}
    if give_json:
        jsons = {}
    for index in match_df.index[start_index:end_index]:
        current_id = str(match_df.loc[index]["gameId"])
        print(current_id)
        matches[current_id], ppi = get_match(current_id)
        result = get_timeline(current_id, ppi, region, give_json)
        dfs[current_id] = result["df"]
        if give_json:
            jsons[current_id] = result["json"]
    if give_json:
        return matches, dfs, jsons
    else:
        return matches, dfs

fetching timelines - I don't have a function to make the calls fit the api limits so you have to pause briefly between each of the following cells - its not bad though.

In [7]:
result1 = get_n_timelines(matches, 0, 40)

4758318885
4758080001
4757537954
4756152547
4755546321
4748107995
4744152493
4742500253
4742168190
4742159411
4669599190
4668932045
4668810801
4665455970
4662458766
4660933260
4658383282
4658159482
4655655968
4651946207
4651011235
4650913342
4650801256
4633492176
4633319621
4631099181
4629920541
4629106507
4625963005
4622647593
4616967917
4616577273
4609450944
4604876381
4598325463
4592845549
4573020600
4555960882
4555921982
4553887990


In [9]:
result2 = get_n_timelines(matches, 40, 80)

4552028319
4544368107
4544372229
4544335028
4542083040
4540074459
4539141730
4537862570
4535839528
4528467940
4528035920
4520522751
4514179696
4509017053
4508910315
4507272154
4503195065
4502791368
4502690200
4502533888
4498692977
4498628202
4498461682
4496965859
4496386394
4494749025
4492823203
4491147223
4491143425
4484596745
4480575656
4478911697
4478771421
4476545133
4470325921
4470371078
4468569705
4468416958
4466385623
4464408055


In [10]:
result3 = get_n_timelines(matches, 80, 120)

4456274649
4456185422
4448956473
4448834078
4448628377
4446420179
4446373912
4446069318
4441819455
4436066469
4434408444
4429689512
4428873490
4428228448
4425852270
4425775790
4424773215
4423982035
4413183949
4410969345
4410511912
4410490837
4409113025
4409065738
4408476991
4397581855
4395118062
4394535536
4392931315
4392746003
4379145543
4379097545
4377445683
4371938229
4364006121
4363890040
4363312004
4361865996
4361599335
4361013090


In [11]:
result4 = get_n_timelines(matches, 80, 120)

4456274649
4456185422
4448956473
4448834078
4448628377
4446420179
4446373912
4446069318
4441819455
4436066469
4434408444
4429689512
4428873490
4428228448
4425852270
4425775790
4424773215
4423982035
4413183949
4410969345
4410511912
4410490837
4409113025
4409065738
4408476991
4397581855
4395118062
4394535536
4392931315
4392746003
4379145543
4379097545
4377445683
4371938229
4364006121
4363890040
4363312004
4361865996
4361599335
4361013090


In [12]:
# rdd = result_df_dis 
rdd = [result1[1],result2[1],result3[1],result4[1]]
result_ids = set(rdd[0].keys())
result_ids.update(result2[1].keys(),result3[1].keys(),result4[1].keys())
len(result_ids)

120

In [13]:
dfs_dict = {**rdd[0], **rdd[1], **rdd[2], **rdd[3]}

In [14]:
len(dfs_dict.keys())

120

In [15]:
import pickle
pickle.dump(dfs_dict, open("data/timeline-di.p", "wb"))

In [16]:
rjd = [result1[2],result2[2],result3[2],result4[2]]
jsons_dict = {**rjd[0], **rjd[1], **rjd[2], **rjd[3]}

In [17]:
pickle.dump(jsons_dict, open("data/json-timeline-di.p", "wb"))

In [18]:
rmd = [result1[0],result2[0],result3[0],result4[0]]
match_dict = {**rmd[0], **rmd[1], **rmd[2], **rmd[3]}

In [19]:
pickle.dump(match_dict, open("data/match-di.p", "wb"))